## 토픽모델링

사용할 데이터 : 멜론 사이트를 크롤링한 1990년부터 연도별 인기있던 Top 30 노래의 가사 데이터

In [1]:
# Data Preprocessing Package
import re                       #정규식
import numpy as np
import pandas as pd
import os                      #디렉토리와 경로정보

# NLP Package
from konlpy.tag import * 
import gensim                    #토픽모델링을 하는 라이브러리
import gensim.corpora as corpora #텍스트분석
from gensim.models import CoherenceModel
from collections import Counter

# Visualization Package   #LDA시각화
import pyLDAvis 
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline


from pprint import pprint #pretty print
import itertools #iterable 객체 처리
import math

import logging #로그처리
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
 
import warnings #경고 무시
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
melon_1990s = pd.concat([pd.read_csv('멜론1990.csv'), pd.read_csv('멜론1991.csv'), pd.read_csv('멜론1992.csv'), pd.read_csv('멜론1993.csv'),
          pd.read_csv('멜론1994.csv'), pd.read_csv('멜론1995.csv'), pd.read_csv('멜론1996.csv'), pd.read_csv('멜론1997.csv'),
          pd.read_csv('멜론1998.csv'), pd.read_csv('멜론1999.csv')])

melon_2000s = pd.concat([pd.read_csv('멜론2000.csv'), pd.read_csv('멜론2001.csv'), pd.read_csv('멜론2002.csv'), pd.read_csv('멜론2003.csv'),
          pd.read_csv('멜론2004.csv'), pd.read_csv('멜론2005.csv'), pd.read_csv('멜론2006.csv'), pd.read_csv('멜론2007.csv'),
          pd.read_csv('멜론2008.csv'), pd.read_csv('멜론2009.csv')])

melon_2010s = pd.concat([pd.read_csv('멜론2010.csv'), pd.read_csv('멜론2011.csv'), pd.read_csv('멜론2012.csv'), pd.read_csv('멜론2013.csv'),
          pd.read_csv('멜론2014.csv'), pd.read_csv('멜론2015.csv'), pd.read_csv('멜론2016.csv'), pd.read_csv('멜론2017.csv'),
          pd.read_csv('멜론2018.csv'), pd.read_csv('멜론2019.csv')])

melon_2020s = pd.concat([pd.read_csv('멜론2020.csv'), pd.read_csv('멜론2021.csv')])

melon = pd.concat([melon_1990s, melon_2000s, melon_2010s, melon_2020s])

In [3]:
melon

,제목,가수,가사
0,희망사항,변진섭,청바지가 잘 어울리는 여자 밥을 많이 먹어도 배 안나오는 여자 내 얘기가 재미 없어...
1,사랑일뿐야,김민우,나를 어떻게 생각하냐고 너는 내게 묻지만 대답하기는 힘들어 너에게 이런 얘길 한다면...
2,유리창엔 비,햇빛촌,낮부터 내린 비는 이 저녁 유리창에 이슬만 뿌려놓고서 밤이 되면 더욱 커지는 시계소...
3,비오는 날 수채화,"김현식, 강인원, 권인하",빗방울 떨어지는 그 거리에 서서 그대 숨소리 살아있는 듯 느껴지면 깨끗한 붓 하나를...
4,마지막 콘서트,이승철,지금 슬픈 내 모습은 무대뒤 한 소녀 애써 눈물 참으며 바라보고 있네 무대 뒤에 그...
...,...,...,...
25,"모든 날, 모든 순간 (Every day, Every Moment)",폴킴,네가 없이 웃을 수 있을까 생각만 해도 눈물이나 힘든 시간 날 지켜준 사람 이제는 ...
26,"LOVE DAY (2021) (바른연애 길잡이 X 양요섭, 정은지)","양요섭, 정은지",참 많이 궁금해 전부 다 궁금해 왜 잠이 안 오고 네 얼굴만 보여 나도 궁금해 이 ...
27,나랑 같이 걸을래 (바른연애 길잡이 X 적재),적재,가을밤이 찾아와 그대를 비추고 또 나를 감싸네 눈을 감을 때마다 향기로운 네 맘이 ...
28,작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey),방탄소년단,모든 게 궁금해 How’s your day Oh tell me 뭐가 널 행복하게 하...


In [4]:
gasa = melon.가사

In [5]:
# Null 값이 존재하는지 확인
print('Null값이 있는지 확인..', gasa.isnull().values.any())

gasa = gasa.dropna(how = 'any') # Null 값이 존재하는 행 제거
print('Null값이 있는지 확인..', gasa.isnull().values.any())
gasa = gasa.reset_index(drop=True)

Null값이 있는지 확인.. True
Null값이 있는지 확인.. False


In [6]:
gasa

0      청바지가 잘 어울리는 여자 밥을 많이 먹어도 배 안나오는 여자 내 얘기가 재미 없어...
1      나를 어떻게 생각하냐고 너는 내게 묻지만 대답하기는 힘들어 너에게 이런 얘길 한다면...
2      낮부터 내린 비는 이 저녁 유리창에 이슬만 뿌려놓고서 밤이 되면 더욱 커지는 시계소...
3      빗방울 떨어지는 그 거리에 서서 그대 숨소리 살아있는 듯 느껴지면 깨끗한 붓 하나를...
4      지금 슬픈 내 모습은 무대뒤 한 소녀 애써 눈물 참으며 바라보고 있네 무대 뒤에 그...
                             ...                        
943    네가 없이 웃을 수 있을까 생각만 해도 눈물이나 힘든 시간 날 지켜준 사람 이제는 ...
944    참 많이 궁금해 전부 다 궁금해 왜 잠이 안 오고 네 얼굴만 보여 나도 궁금해 이 ...
945    가을밤이 찾아와 그대를 비추고 또 나를 감싸네 눈을 감을 때마다 향기로운 네 맘이 ...
946    모든 게 궁금해 How’s your day Oh tell me 뭐가 널 행복하게 하...
947    STAYC girls it’s going down Time is running bo...
Name: 가사, Length: 948, dtype: object

In [7]:
print("처리할 데이터수 : ",len(gasa))

처리할 데이터수 :  948


### 토크나이징, 불용어 처리, 말뭉치 생성, 빈도 계수

In [8]:
stopword_vocab = './stopword.txt' # 불용어 파일 불러오기
tokenizer = Okt() # 토큰나이저 지정
sep = "\n" # 불용어 처리 인자

In [9]:
def build_vocab(data_frame ,stopword_vocab, separate):
    
    # 불용어 데이터를 가져와 리스트로 변환합니다.
    with open(stopword_vocab, encoding = 'utf-8') as f:
        temp1 = []
        for i in f:
            temp1.append(i)
            
    globals()['stopword_vocab'] = []
    
    # 불용어 데이터는 전역변수 stopword_vocab 선언합니다. 
    # 구분자에 따라 stopword_vocab에 추가하여 불용어 사전을 구축합니다.
    for j in range(len(temp1)):
        temp2 = temp1[j].rstrip(separate)
        globals()['stopword_vocab'].append(temp2)
    
    # okt token에서 명사만 출력합니다. 단어의 길이가 1 초과인 단어만 출력합니다. 
    globals()['list_sent2words'] =[]
    for i in range(len(data_frame)) :
        num_list=[]
        temp = tokenizer.nouns(data_frame[i])
        for j in range(len(temp)):

            if len(temp[j]) > 1:
                num_list.append(temp[j])
        globals()['list_sent2words'].append(num_list)
    
    return [[word for word in doc if word not in globals()['stopword_vocab']] for doc in globals()['list_sent2words']]

result_data = build_vocab(gasa, stopword_vocab, sep)

In [10]:
# 불용어를 제외하고 단어의 길이가 2 이상인 명사만 저장된 리스트
result_data

[['청바지',
  '여자',
  '여자',
  '얘기',
  '재미',
  '여자',
  '여자',
  '머리',
  '무스',
  '윤기',
  '여자',
  '고요한',
  '눈빛',
  '시력',
  '여자',
  '김치',
  '볶음밥',
  '여자',
  '목젖',
  '여자',
  '마음',
  '만날',
  '여자',
  '내지',
  '여자',
  '소리',
  '안나',
  '여자',
  '다리',
  '치마',
  '여자',
  '위로',
  '여자',
  '만난',
  '이후',
  '미팅',
  '한번',
  '한번',
  '여자',
  '라라라라',
  '라라라라',
  '라라라',
  '라라라라',
  '라라라라',
  '라라라라',
  '라라라라',
  '라라라라',
  '라라라라',
  '라라라라',
  '라라라',
  '라라라라',
  '라라라라',
  '라라라',
  '라라라라',
  '라라라라',
  '라라라라',
  '라라라라',
  '라라라라',
  '라라라라',
  '라라라라',
  '라라라',
  '여자',
  '여자',
  '희망사항',
  '정말',
  '여자',
  '남자'],
 ['생각',
  '대답',
  '얘길',
  '표정',
  '골목길',
  '외등',
  '외로움',
  '그대',
  '위해',
  '이별',
  '그대',
  '온몸',
  '사랑',
  '생각',
  '대답',
  '얘길',
  '표정',
  '골목길',
  '외등',
  '외로움',
  '그대',
  '위해',
  '이별',
  '그대',
  '온몸',
  '사랑'],
 ['저녁',
  '유리창',
  '이슬',
  '더욱',
  '시계',
  '소리',
  '마음',
  '빗줄기',
  '아주',
  '헤메',
  '우산',
  '저녁',
  '유리창',
  '슬픔',
  '뿌리',
  '마음속',
  '기억',
  '빗줄기',
  '기억',
  '순간',
  '사이',
  '마음',
  '두운',
  '우산',
  '저녁',


In [11]:
# 전체 에 대한 워드 카운트 계수 확인

def word_corpus(result_data):
    #전체 단어의 갯수 파악
    words = list(itertools.chain(*result_data))
    print('전체 워드의 개수 : {}'.format(len(words)))

    #단어의 빈도수를 확인 후 추가할 불용어 확인 작업
    vocab = Counter(words)
    vocab_size = len(words)
    vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 n개의 단어만 저장 vocab
    return vocab

vocab=word_corpus(result_data)

# 전체 워드의 빈도 계수 
df_corpus=pd.DataFrame(columns=["text","count"])
tmp_list=[]
tmp_list1=[]
for word, num in vocab:
    tmp_list.append(word)
    tmp_list1.append(num)
df_corpus['text']=tmp_list
df_corpus['count']=tmp_list1
#상위 20개의 워드 카운드 계수만 출력
a=df_corpus.head(1000)
print(df_corpus.head(20))


# 토픽 모델링 딕셔너리 생성
id2word = corpora.Dictionary(result_data)
 
# 토픽모델링에 사용할 말뭉치 생성
texts = result_data
 
# 용어-문서 빈도
corpus = [id2word.doc2bow(text) for text in texts]

전체 워드의 개수 : 41384
   text  count
0    사랑   2803
1    그대   1245
2    사람    747
3    다시    694
4    눈물    601
5    마음    581
6    이제    531
7    생각    527
8    세상    499
9    모습    463
10   지금    441
11   가슴    402
12   오늘    398
13   기억    348
14   모든    332
15   정말    297
16   하루    290
17   보고    286
18   위해    268
19   이별    266


빈도수가 너무 많은 '사랑', '그대'를 stopwords에 넣고 다시 토크나이징 진행 ( 토픽 분류에 해가 될 수 있음 )

In [12]:
stopword_vocab = './stopword2.txt' # 불용어 파일 불러오기
tokenizer = Okt() # 토큰나이저 지정
sep = "\n" # 불용어 처리 인자

In [13]:
def build_vocab(data_frame ,stopword_vocab, separate):
    
    # 불용어 데이터를 가져와 리스트로 변환합니다.
    with open(stopword_vocab, encoding = 'utf-8') as f:
        temp1 = []
        for i in f:
            temp1.append(i)
            
    globals()['stopword_vocab'] = []
    
    # 불용어 데이터는 전역변수 stopword_vocab 선언합니다. 
    # 구분자에 따라 stopword_vocab에 추가하여 불용어 사전을 구축합니다.
    for j in range(len(temp1)):
        temp2 = temp1[j].rstrip(separate)
        globals()['stopword_vocab'].append(temp2)
    
    # okt token에서 명사만 출력합니다. 단어의 길이가 1 초과인 단어만 출력합니다. 
    globals()['list_sent2words'] =[]
    for i in range(len(data_frame)) :
        num_list=[]
        temp = tokenizer.nouns(data_frame[i])
        for j in range(len(temp)):

            if len(temp[j]) > 1:
                num_list.append(temp[j])
        globals()['list_sent2words'].append(num_list)
    
    return [[word for word in doc if word not in globals()['stopword_vocab']] for doc in globals()['list_sent2words']]

result_data = build_vocab(gasa, stopword_vocab, sep)

In [14]:
# 전체 에 대한 워드 카운트 계수 확인

def word_corpus(result_data):
    #전체 단어의 갯수 파악
    words = list(itertools.chain(*result_data))
    print('전체 워드의 개수 : {}'.format(len(words)))

    #단어의 빈도수를 확인 후 추가할 불용어 확인 작업
    vocab = Counter(words)
    vocab_size = len(words)
    vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 n개의 단어만 저장 vocab
    return vocab

vocab=word_corpus(result_data)

# 전체 워드의 빈도 계수 
df_corpus=pd.DataFrame(columns=["text","count"])
tmp_list=[]
tmp_list1=[]
for word, num in vocab:
    tmp_list.append(word)
    tmp_list1.append(num)
df_corpus['text']=tmp_list
df_corpus['count']=tmp_list1
#상위 20개의 워드 카운드 계수만 출력
a=df_corpus.head(1000)
print(df_corpus.head(20))


# 토픽 모델링 딕셔너리 생성
id2word = corpora.Dictionary(result_data)
 
# 토픽모델링에 사용할 말뭉치 생성
texts = result_data
 
# 용어-문서 빈도
corpus = [id2word.doc2bow(text) for text in texts]

전체 워드의 개수 : 37336
   text  count
0    사람    747
1    다시    694
2    눈물    601
3    마음    581
4    이제    531
5    생각    527
6    세상    499
7    모습    463
8    지금    441
9    가슴    402
10   오늘    398
11   기억    348
12   모든    332
13   정말    297
14   하루    290
15   보고    286
16   위해    268
17   이별    266
18   추억    246
19   하늘    238


### 토픽 평가

In [15]:
# coherence score가 nan 값이 나오는 오류 제거
# https://github.com/RaRe-Technologies/gensim/issues/3040

from gensim.topic_coherence import direct_confirmation_measure

log = logging.getLogger(__name__)

ADD_VALUE = 1


def custom_log_ratio_measure(segmented_topics, accumulator, normalize=False, with_std=False, with_support=False):
    topic_coherences = []
    num_docs = float(accumulator.num_docs)
    for s_i in segmented_topics:
        segment_sims = []
        for w_prime, w_star in s_i:
            w_prime_count = accumulator[w_prime]
            w_star_count = accumulator[w_star]
            co_occur_count = accumulator[w_prime, w_star]

            if normalize:
                # For normalized log ratio measure
                numerator = custom_log_ratio_measure([[(w_prime, w_star)]], accumulator)[0]
                co_doc_prob = co_occur_count / num_docs
                m_lr_i = numerator / (-np.log(co_doc_prob + direct_confirmation_measure.EPSILON))
            else:
                # For log ratio measure without normalization
                ### _custom: Added the following 6 lines, to prevent a division by zero error.
                if w_star_count == 0:
                    log.info(f"w_star_count of {w_star} == 0. Adding {ADD_VALUE} to the count to prevent error. ")
                    w_star_count += ADD_VALUE
                if w_prime_count == 0:
                    log.info(f"w_prime_count of {w_prime} == 0. Adding {ADD_VALUE} to the count to prevent error. ")
                    w_prime_count += ADD_VALUE
                numerator = (co_occur_count / num_docs) + direct_confirmation_measure.EPSILON
                denominator = (w_prime_count / num_docs) * (w_star_count / num_docs)
                m_lr_i = np.log(numerator / denominator)

            segment_sims.append(m_lr_i)

        topic_coherences.append(direct_confirmation_measure.aggregate_segment_sims(segment_sims, with_std, with_support))

    return topic_coherences

In [16]:
from gensim.topic_coherence import direct_confirmation_measure
# from my_custom_module import custom_log_ratio_measure

direct_confirmation_measure.log_ratio_measure = custom_log_ratio_measure

In [17]:
# Perplexity(혼란도) 확률 모델이 결과를 얼마나 정확하게 예측하는지.낮을수록 정확하게 예측
# Coherence Score 을 판단, 토픽이 얼마나 의미론적으로 일관성 있는지, 높을수록 의미론적 일관성 높음


#NUM_TOPICS = int(input('토픽의 개수를 입력해 주세요. '))
#TOPICS_W_NUM = int(input('출력할 토픽별 단어의 개수를 입력해 주세요 '))
#save_lda_model= int(input("선택한 토픽 모델을 저장하시겠습니까? \n0 저장  \n1 미저장  "))

TOPICS_W_NUM =20 # 출력할 토픽별 단어의 개수
save_lda_model=0
RANDOM_STATE = 2020
UPDATE_EVERY = 1
CHUNKSIZE = 100
PASSES = 10
ALPHA = 'auto'
PER_WORD_TOPICS = True
print('NUM_TOPICS', 'perplexity', 'coherence')
for i in range(1,30):
    NUM_TOPICS=i
 
  #해당 셀은 토픽모델링(LDA)에 대해 모델을 정의하는 셀입니다.
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, 
                                              num_topics=NUM_TOPICS, random_state=RANDOM_STATE, 
                                              update_every=UPDATE_EVERY, chunksize=CHUNKSIZE,
                                              passes=PASSES, alpha=ALPHA, per_word_topics=PER_WORD_TOPICS)

    # 토픽 출력
    #  pprint(lda_model.print_topics(num_words=TOPICS_W_NUM))
    doc_lda = lda_model[corpus]

    # 모델 저장 
    # if save_lda_model == 0:
        #lda_model.save('LDA_MODEL_SAVE')
    # 0번 토픽,- 중요단어들이 가중치 순으로 나옴(20개)
    """
    해당 셀은 설계한 모델을 계산하는 셀입니다.
    측정은 Perplexity와 Coherence Score입니다.
    """
    #print('\nNUM_TOPICS',NUM_TOPICS)
    # Perplexity 
    #print('Perplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.

    # Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=result_data, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    #print('Coherence Score: ', coherence_lda)
    # print('NUM_TOPICS',NUM_TOPICS,'Perplexity: ', lda_model.log_perplexity(corpus),'Coherence: ', coherence_lda)

    print('T',NUM_TOPICS, lda_model.log_perplexity(corpus), coherence_lda)

NUM_TOPICS perplexity coherence
T 1 -6.87798497364969 0.16814810731568822
T 2 -6.80683857922484 0.3650628951857735
T 3 -6.811711024251917 0.40089532417757856
T 4 -6.8195504918407135 0.41365555189781006
T 5 -6.846872823474518 0.41065102765394706
T 6 -6.842461568817659 0.37968887157931946
T 7 -6.850536993737264 0.3950074443621805
T 8 -6.8565716069197284 0.42942945945052236
T 9 -6.875261590223203 0.4173685197721107
T 10 -6.98075600704711 0.4195561203236153
T 11 -7.211746405930649 0.4288569030584799
T 12 -7.573751604752442 0.4637080579792366
T 13 -8.156873568799886 0.40657332004238156
T 14 -9.070103592785902 0.4171872041092636
T 15 -10.214528327757895 0.38482561431459356
T 16 -11.201443155149475 0.3765989554247665
T 17 -11.905399373801753 0.40473600369989227
T 18 -12.237122958717189 0.436772247171873
T 19 -12.545068721606839 0.3726188550483327
T 20 -12.858202622257108 0.3789378340697015
T 21 -13.161998237540676 0.41230905138082957
T 22 -13.493674505593482 0.3972559514711938
T 23 -13.782925

#### perplexity가 가장 작은 29개의 topic

In [19]:
save_lda_model=0
RANDOM_STATE = 2020
UPDATE_EVERY = 1
CHUNKSIZE = 100
PASSES = 10
ALPHA = 'auto'
PER_WORD_TOPICS = True
print('NUM_TOPICS', 'perplexity', 'coherence')

NUM_TOPICS=29

#해당 셀은 토픽모델링(LDA)에 대해 모델을 정의하는 셀입니다.
lda_model29 = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, 
                                          num_topics=NUM_TOPICS, random_state=RANDOM_STATE, 
                                          update_every=UPDATE_EVERY, chunksize=CHUNKSIZE,
                                          passes=PASSES, alpha=ALPHA, per_word_topics=PER_WORD_TOPICS)


# Coherence Score
coherence_model_lda29 = CoherenceModel(model=lda_model29, texts=result_data, dictionary=id2word, coherence='c_v')
coherence_lda29 = coherence_model_lda29.get_coherence()


print('T',NUM_TOPICS, lda_model29.log_perplexity(corpus), coherence_lda29)

NUM_TOPICS perplexity coherence
T 29 -15.664695718550174 0.39944392491331926


#### 토픽별 키워드 조회

In [20]:
NUM_TOPICS=29

for topic_id in range(NUM_TOPICS):
    topic_word_probs29 = lda_model29.show_topic(topic_id, TOPICS_W_NUM)
    print("Topic ID: {}".format(topic_id))

    for topic_word, prob in topic_word_probs29:
        print("\t{}\t{}".format(topic_word, prob))
    print("\n")

Topic ID: 0
	발걸음	0.17727990448474884
	예전	0.16440509259700775
	한숨	0.12651371955871582
	갈수	0.008217444643378258
	상일	0.004070512484759092
	질때	0.003035296220332384
	온도	0.00013769084762316197
	신호등	0.00013769084762316197
	푸른색	0.00013769084762316197
	머릿속	0.00013769084762316197
	나라	0.00013769084762316197
	은색	0.00013769084762316197
	노란색	0.00013769084762316197
	야야	0.00013769084762316197
	자국	0.00013769084762316197
	비행운	0.00013769084762316197
	샛노랄	0.00013769084762316197
	서해	0.00013769083307124674
	난감	0.00013769083307124674
	날로	0.00013769083307124674


Topic ID: 1
	강북	0.0002661698090378195
	안달	0.0002661698090378195
	허파	0.0002661698090378195
	열광	0.0002661698090378195
	날자	0.0002661698090378195
	지상	0.0002661698090378195
	드네	0.0002661698090378195
	행실	0.0002661698090378195
	유행가	0.0002661698090378195
	싸구려	0.0002661698090378195
	술버릇	0.0002661698090378195
	날로	0.0002661698090378195
	난감	0.0002661698090378195
	끄덕	0.0002661698090378195
	고도	0.0002661698090378195
	효과	0.0002661698090378195
	톡톡	0.00026616980903781

- 토픽 0 : 온도, 푸른색, 은색, 노란색, 샛노랄 등 색깔과 관련된 단어들이 위치해있다. ( 생각나는 노래 : 비행운, 신호등 )
- 토픽 1 : 유행가, 싸구려, 술버릇, 아빠 등 조금 올드한 느낌이 드는 단어들, 아버지 세대의 느낌이 드는 단어들이 위치해있다.
- 토픽 2 : 겨울, 풍경, 유리창, 창가, 등불, 방황 등 감성적이고 차가운 느낌이 드는 단어들이 위치해있다.
- 토픽 3 : 사람, 눈빛, 얼굴, 표정, 가슴 등 사람의 신체적인 특징을 나타내는 단어들이 위치해있다.
- 토픽 4 : 토픽 1과 같은 단어들이 위치해 있다. ( ?? ) 
- 토픽 5 : 마음, 향기, 저녁, 눈물 등 저녁 감성이 느껴지는 단어들이 엿보인다.
- 토픽 6 : 거짓, 슬픔, 심장, 외톨이, 치료 등 외로움이 느껴지는 슬픈 단어들이 위치해있다.
- 토픽 7 : 라며, 란걸, 하리, 상해 등 주로 어미에 오는 단어들이 들어가 있고 이를 stopwords에 추가할 경우 더 좋은 성능을 기대할 수 있을 것 같다.
- 토픽 8 : 오늘, 하루, 어제, 십년 등 주로 기간을 나타내는 단어들이 많이 보인다.
- 토픽 9 : 이유를 주제로한 단어들이 모인 것으로 보이지만 큰 특징은 보이지 않는다. 
- 토픽 10 : 바보, 멍하니, 만남같은 주로 기다림을 주제로 한 노래에 등장할 법한 가사들이 위치해있다.
- 토픽 11 : 정말, 제발, 사실같은 부사가 위치해있다.
- 토픽 12 : 크게 통일되는 주제를 찾기가 어렵다.
- 토픽 13 : 항상, 내일, 미래, 기쁨, 웃음 등 긍정적인 느낌이 드는 단어들이 위치해있다.
- 토픽 14 : 입술, 매력, 키스, 소녀 등 설렘이 느껴지고 아름다운 단어들이 위치해있다.
- 토픽 15 : 현실, 글썽, 사연, 세상 등 조금 현실적이고 슬픈느낌이 드는 단어들이 위치해있다.
- 토픽 16 : 취해, 엄마, 마음속 침묵 등 술과 함께 누군가를 상상하거나 그리운 사람을 떠올리는 느낌이 드는 단어들이 위치해있다.
- 토픽 17 : 아픔, 새벽, 외로움, 그리움만 등 외롭고 그리움을 나탄는 단어들이 위치해있다.
- 토픽 18 : 세상이라는 단어가 주를 이루고 있고 따라서 다른 단어들은 장미꽃, 희망 같이 긍정적인 단어, 거짓말 같은 부정적인 단어가 혼재한다.
- 토픽 19 : 우린, 서로, 장난 등 연인 혹은 친구와 같이 있을 때 사용되는 단어들이 위치해있다.
- 토픽 20 : 기억, 상처, 추억 등 과거에 대한 생각이 떠오르는 단어들이 위치해있다.
- 토픽 21 : 그녀, 여름, 용기, 치마 등 여성과 관련된 설레는 단어들이 위치해있다.
- 토픽 22 : 만날, 인사, 기다림 등 누군가와의 만남, 혹은 기다림이 나타나는 단어들이 위치해있다.
- 토픽 23 : 친구가 주를 이루고 있고 토픽 0에서 등장했던 색깔과 관련된 단어들이 위치해있다.
- 토픽 24 : 노래, 바람이 주를 이루고 있고 자연과 관련된 단어들이 위치해있다.
- 토픽 25 : 고백, 오늘밤, 부탁 같이 고백하는 노래에 사용될법한 단어들이 위치해있다.
- 토픽 26 : 소리, 바다, 파도 등 여름이 생각나는 바닷가 단어들이 위치해있다.
- 토픽 27 : 눈물, 가슴 등 이전 토픽들에도 들어가있던 단어들이 위치해있다.
- 토픽 28 : 남자, 여자 단어들과 주로 뜻을 갖지 않는 어미들이 위치해있다. 

**아무래도 가사의 노래가 사랑 혹은 슬픔(이별) 노래가 많다 보니 토픽들도 두가지의 큰 주제로 구분되어지는 느낌을 받았다. 또한 토픽1, 토픽4와 같이 완전 같은 단어들이 분류된 케이스, 토픽12와 같이 크게 주제를 잡기 어렵게 분류된 경우도 존재했다.** 

#### coherence score가 가장 높은 12개의 토픽의 결과!!

In [21]:
# NUM_TOPICS = 12로 재 학습

TOPICS_W_NUM =20 # 출력할 토픽별 단어의 개수
save_lda_model=0
RANDOM_STATE = 2020
UPDATE_EVERY = 1
CHUNKSIZE = 100
PASSES = 10
ALPHA = 'auto'
PER_WORD_TOPICS = True
print('NUM_TOPICS', 'perplexity', 'coherence')

NUM_TOPICS=12

#해당 셀은 토픽모델링(LDA)에 대해 모델을 정의하는 셀입니다.
lda_model12 = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, 
                                          num_topics=NUM_TOPICS, random_state=RANDOM_STATE, 
                                          update_every=UPDATE_EVERY, chunksize=CHUNKSIZE,
                                          passes=PASSES, alpha=ALPHA, per_word_topics=PER_WORD_TOPICS)


# Coherence Score
coherence_model_lda12 = CoherenceModel(model=lda_model12, texts=result_data, dictionary=id2word, coherence='c_v')
coherence_lda12 = coherence_model_lda12.get_coherence()


print('T',NUM_TOPICS, lda_model12.log_perplexity(corpus), coherence_lda12)

NUM_TOPICS perplexity coherence
T 12 -7.573751604752442 0.4637080579792366


#### 토픽별 키워드 조회

In [22]:
NUM_TOPICS = 12

for topic_id in range(NUM_TOPICS):
    topic_word_probs12 = lda_model12.show_topic(topic_id, TOPICS_W_NUM)
    print("Topic ID: {}".format(topic_id))

    
    for topic_word, prob in topic_word_probs12:
        print("\t{}\t{}".format(topic_word, prob))
    print("\n")

Topic ID: 0
	그녀	0.0929611474275589
	밤하늘	0.038122132420539856
	벌써	0.035805173218250275
	하나로	0.024322424083948135
	송이	0.023669792339205742
	베이비	0.022097760811448097
	다운타운	0.022097760811448097
	으르렁	0.02204214595258236
	고백	0.019669337198138237
	그림자	0.019368356093764305
	야야	0.01899593323469162
	꽃잎	0.018342584371566772
	마주	0.01831807568669319
	용기	0.01747623272240162
	소식	0.016938503831624985
	장미꽃	0.01390683464705944
	새벽	0.013855669647455215
	길이	0.013608208857476711
	달이	0.013573571108281612
	외치	0.013087167404592037


Topic ID: 1
	매일	0.14634287357330322
	그땐	0.03544526919722557
	하나요	0.029888221994042397
	어서	0.029080504551529884
	대도	0.027957595884799957
	물보라	0.02693706378340721
	취해	0.023344894871115685
	제일	0.021215813234448433
	우연	0.02110127918422222
	진짜	0.018253756687045097
	먼저	0.01518103014677763
	온통	0.014657323248684406
	시선	0.01403206866234541
	히라기	0.013483995571732521
	루루	0.012351338751614094
	행동	0.010682424530386925
	오지	0.010355481877923012
	여행	0.00961446762084961
	나인	0.009506963193416595
	불

- 토픽 0 : 여러 단어들이 0.1%가 안되는 낮은 비중을 보이며 그녀, 고백, 꽃잎같이 설렘을 느낄 수 있는 단어들이 위치해있다.
- 토픽 1 : 매일, 그땐, 제일 등 명사를 앞에서 꾸며주는 단어들이 위치해있다.
- 토픽 2 : 겨울, 바다, 파도, 여름 등 계절과 바다가 떠오르는 단어들이 위치해있다.
- 토픽 3 : 마음, 생각, 가슴 등 누군가를 기억하거나 떠올리는 데 사용되는 단어들이 위치해있다.
- 토픽 4 : 점핑, 빠빠빠, 외톨이야와 같이 특정 노래가 떠오르는 단어들이 엿보인다.
- 토픽 5 : 남자, 여자, 오빠와 같이 성별이나 머리, 정신같이 생각과 관련된 단어들이 위치해있다.
- 토픽 6 : 무릎, 어깨같은 신체부위, 담배, 샴푸와 같은 생활용품 단어들이 위치해있다.
- 토픽 7 : 다시, 세상, 하늘같이 노래 가사에 많이 사용될법한 단어들이 많이 보인다.
- 토픽 8 : 보고, 운명, 걱정 등 누군가를 기다리는데 사용될만한 단어들이 위치해있다.
- 토픽 9 : 노래, 가사 등과 후렴구에 나올법한 단어들이 많이 위치해있다.
- 토픽 10 : 향기, 세월, 바람과 같이 지나가버린 세월, 인연 등이 느껴지는 단어들이 엿보인다.
- 토픽 11 : 달라, 살짝, 벚꽃과 같이 희망이나 포기하지 않는 느낌이 드는 단어들이 위치해있다.

**토픽의 개수가 29개였던 위의 모델보다 한 토픽을 지배하고 있는 단어가 없고 prob가 잘 나뉘어져 있는 모습을 확인할 수 있다. 또한 위의 모델에서는 같은 단어들이 2개의 토픽에서 보였던 경우도 있었는데 그러한 경우도 없는 것으로 보아 토픽의 개수를 coherence를 기준으로 12개로 나누었을 때가 perplexity를 기준으로 나누었을 때보다 더 잘 구분하였다고 판단하였다.**

### 시각화

In [23]:
# perplexity가 가장 낮은 29개의 Topic

pyLDAvis.enable_notebook()
def create_vis(model):
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim_models.prepare(model, corpus, id2word, sort_topics=False)
    # pyLDAvis.save_html(vis, './Result_lda_vis.html')
    return vis
#lda_model or optimal_model
create_vis(lda_model29)

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.140319  0.005674       1        1   0.501411
1      0.161554  0.006671       2        1   0.029964
2      0.098248  0.006642       3        1   1.203890
3     -0.288028 -0.182204       4        1  14.514866
4      0.161554  0.006671       5        1   0.030110
5      0.007553 -0.017946       6        1   3.015114
6      0.092348  0.002736       7        1   1.182396
7     -0.090748 -0.034965       8        1   2.940517
8     -0.159159 -0.249536       9        1   5.597964
9      0.130673  0.005014      10        1   0.593018
10     0.054236 -0.004030      11        1   1.925855
11    -0.042724  0.041427      12        1   3.277781
12     0.105395  0.002835      13        1   0.930183
13    -0.133895 -0.081877      14        1   4.274529
14     0.052795  0.005293      15        1   1.879200
15     0.075423 -0.004104      16        1   1.843004
16    -0.077446  0.041014      17        1   4.045215
17     0.096530  0.001928      18        1   1.361197
18    -0.036396 -0.063416      19        1   4.173865
19     0.023132 -0.015673      20        1   2.341447
20    -0.171600  0.279252      21        1   5.827300
21     0.063662  0.007066      22        1   1.871256
22     0.049251  0.001855      23        1   1.532513
23     0.141171  0.007081      24        1   0.635738
24     0.006967 -0.009229      25        1   2.342918
25     0.037943 -0.007673      26        1   1.832653
26    -0.119876  0.046414      27        1   4.551412
27    -0.351059  0.154671      28        1  22.239542
28    -0.027823  0.048411      29        1   3.505143, topic_info=    Term         Freq        Total Category  logprob  loglift
8     마음   686.000000   686.000000  Default  30.0000  30.0000
67    세상   650.000000   650.000000  Default  29.0000  29.0000
64    사람   906.000000   906.000000  Default  28.0000  28.0000
171   오늘   568.000000   568.000000  Default  27.0000  27.0000
154   다시  1023.000000  1023.000000  Default  26.0000  26.0000
..   ...          ...          ...      ...      ...      ...
28    한번     0.049796   252.842603  Topic29 -10.1766  -5.1817
157   시절     0.049795    32.201217  Topic29 -10.1766  -3.1209
25    정말     0.049791   356.981404  Topic29 -10.1767  -5.5267
64    사람     0.049791   906.992435  Topic29 -10.1767  -6.4591
23    이후     0.049790    47.306496  Topic29 -10.1767  -3.5056

[1159 rows x 6 columns], token_table=      Topic      Freq Term
term                      
237       8  0.973222  가까이
152       4  0.683755   가끔
152       9  0.305508   가끔
492      12  0.879732   가나
179      21  0.413460  가다가
...     ...       ...  ...
205      28  0.861186   후회
620      13  0.402399   휴일
551      14  0.933629   흔적
166      21  0.952468   흠뻑
638      19  0.956035   희망

[572 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [24]:
# coherence가 가장 높은 12개의 Topic

create_vis(lda_model12)

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.098019  0.057493       1        1   3.513744
1      0.090688 -0.021800       2        1   4.124475
2      0.082210 -0.067113       3        1   3.389582
3     -0.347625 -0.099319       4        1  40.871951
4      0.098636  0.044275       5        1   3.623154
5     -0.057758  0.280428       6        1   7.442820
6      0.102701  0.055082       7        1   1.623443
7     -0.324705  0.066580       8        1  19.644066
8      0.025534 -0.112009       9        1   5.566481
9      0.113010  0.034224      10        1   1.692640
10     0.096364 -0.005863      11        1   2.730191
11     0.022924 -0.231978      12        1   5.777453, topic_info=    Term        Freq       Total Category  logprob  loglift
230   보고  309.000000  309.000000  Default  30.0000  30.0000
391   자꾸  243.000000  243.000000  Default  29.0000  29.0000
154   다시  671.000000  671.000000  Default  28.0000  28.0000
77    노래  187.000000  187.000000  Default  27.0000  27.0000
664   매일  226.000000  226.000000  Default  26.0000  26.0000
..   ...         ...         ...      ...      ...      ...
279   어제   35.922744   48.441392  Topic12  -4.0951   2.5522
333   영화   26.067870   33.972606  Topic12  -4.4158   2.5864
222   이름   30.302865   85.705388  Topic12  -4.2653   1.8115
196   지나   25.852171   80.367273  Topic12  -4.4241   1.7170
127   바람   24.569897  224.659460  Topic12  -4.4750   0.6381

[471 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1502     12  0.984099   가기
152       4  0.991059   가끔
738       7  0.895773   가도
2123      5  0.918836   가든
440      11  0.957346   가라
...     ...       ...  ...
551       7  0.909513   흔적
166       3  0.912656   흠뻑
638       5  0.945937   희망
3682      2  0.974646  히라기
3644      7  0.896023   힐끔

[510 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

## LSA, LDA

위에서 분석한 결과에 맞게 LSA 분석 또한 TOPIC의 개수를 12개로 나누어 진행!!

In [25]:
import ujson #입력파일이 json형태
from gensim import corpora #gensim 에서 제공하는 패키지
from gensim import models
from gensim.models import CoherenceModel
from collections import Counter #카운터 사용

In [26]:
FEATURE_POSES = ["NNG", "NNP", "XR"] # NNG:일반명사, NNP:고유명사, XR:어근 ( 분석하고자 하는 부분 )

NUM_TOPICS = 12 #토픽의 개수
NUM_TOPIC_WORDS = 10 #하나의 토픽에 포함되는 단어수

#### 필요 함수 정의

In [27]:
# term을 입력. 단어를 입력하면(예,정치), 단어가 여러 토픽에 해당될 수 있는데 각 토픽에서 갖는 가중치를 표시
def print_term_topics(term, dictionary, model):
    word_id = dictionary.token2id[term]   #단어의 아이디 구함
    print(model.get_term_topics(word_id))  
    
# 문서에 대한 토픽가중치를 반복하면서 전체 문서에 대해서 표시
def print_doc_topics(model, corpus): 
    
    for doc_num, doc in enumerate(corpus):
        topic_probs = model[doc]
        print("Doc num: {}".format(doc_num))

        for topic_id, prob in topic_probs:
            print("\t{}\t{}".format(topic_id, prob))
        
        if doc_num == 2:  # 시간 관계상 2번 문서까지만 출력, "0번문서, 1번문서, 2번문서"에 대해서만 해당문서의 토픽가중치를 표시                                     
            break

        print("\n")  

# ★ 모델링 후 각 토픽별로 중요한 단어들을 표시 ( 가장 중요한 부분 !!)
def print_topic_words(model):
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))

        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
        print("\n")

#### LSA 모델 생성

In [52]:
model_LSA = models.lsimodel.LsiModel(corpus, num_topics=NUM_TOPICS,id2word=id2word)

# corpus, topic을 몇개 추출할 것인지, dictionary를 받아 토픽의 개수만큼 LSA로 표시해주세요 ( == 과거의 명칭인 LSI )

In [53]:
print_topic_words(model_LSA)                  # 전체 토픽별 주요 어휘 출력 

Topic ID: 0
	사람	0.3873557607388645
	다시	0.30206630216167457
	눈물	0.26626235596080045
	마음	0.2380730730875773
	이제	0.23028710828128487
	생각	0.22734861780607685
	세상	0.20832188822386538
	모습	0.19685731228177036
	가슴	0.1735636431822518
	오늘	0.1649933813822169


Topic ID: 1
	사람	-0.8423948257783698
	다시	0.2677612538934596
	오늘	0.2224597649806514
	눈물	0.14758434787049024
	보고	0.13072589013664257
	모습	0.12271466114560925
	그냥	-0.0907664417307683
	하루	0.08732585623820192
	이제	0.08603287904706887
	기억	0.07867325136465071


Topic ID: 2
	눈물	0.6439802265013859
	다시	0.32307051470406867
	모습	-0.2288072254244098
	생각	-0.2274462154708241
	오늘	-0.22741562235929635
	지금	-0.1914582740231452
	마음	-0.1699807739869568
	그녀	-0.16002542708744752
	여자	-0.1527276404308987
	가슴	0.12411548438404796


Topic ID: 3
	다시	-0.651899434142256
	눈물	0.4686029430443464
	오늘	0.2426659984144378
	보고	-0.20150133102401765
	세상	0.18267009417909114
	모습	-0.16178193648991854
	아래	0.12192628787536416
	사람	-0.099987027421197
	머리	0.09989048811524426
	여자	0.09682015180

주로 등장하는 단어인 사람, 다시와 같은 단어들은 여러 토픽에 중복되어 등장하는 것을 확인할 수 있습니다. 전체적으로 count횟수가 높은 단어들이 여러 토픽에 중복되어 등장하여 하나의 토픽을 보았을 때 해당 토픽의 내용을 유추하기가 어렵습니다.

In [32]:
print_doc_topics(model_LSA, corpus) 

Doc num: 0
	0	2.8558742341255114
	1	0.05351767997428548
	2	-4.778519426303499
	3	-3.4469198878599765
	4	5.0911313219539025
	5	2.8129744538836223
	6	-4.3035508042969655
	7	1.963995310760771
	8	4.566222633244986
	9	-5.543490785267105
	10	-5.724479210691011
	11	-11.240532664957827


Doc num: 1
	0	0.9710745622219298
	1	0.2358647638094888
	2	-0.3585675124035915
	3	0.07703846204637795
	4	0.19517382828851343
	5	-0.0315318103562949
	6	-0.04067348752317765
	7	-0.21466110200262
	8	-0.415358386405656
	9	-0.01701286076287807
	10	0.4696989975599468
	11	-0.510530684965062


Doc num: 2
	0	1.2443773141541385
	1	0.42937402567807614
	2	0.03822001592468475
	3	-0.06961390501700014
	4	-0.5609359804580087
	5	-0.15925256343536098
	6	0.25821467042719837
	7	-0.6049547644609643
	8	-0.3347834196903094
	9	1.063559556464241
	10	-0.9719732332663618
	11	-0.3713704140284539


이 결과를 보면 하나의 document에서 거의 모든 토픽에 단어를 할당하였기 때문에 위에서 볼 수 있었듯이 토픽마다 등장하는 단어들이 비슷한 것을 알 수 있습니다.

#### LDA 모델로 분석

In [45]:
# 위의 모델에서 per_word_topics를 False로 지정한 모델
lda_model122 = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, 
                                          num_topics=NUM_TOPICS, random_state=RANDOM_STATE, 
                                          update_every=UPDATE_EVERY, chunksize=CHUNKSIZE,
                                          passes=PASSES, alpha=ALPHA)

In [46]:
print_term_topics("사람", id2word, lda_model122)  # 특정 용어의 토픽별 가중치 출력

[(3, 0.044909067)]


사람이라는 단어가 3 토픽에만 들어간 모습

In [47]:
# 특정 토픽의 주요 용어 출력 
print(lda_model122.show_topic(2, topn=10))          
# 토픽 2의 주요 단어 10개 표시( 가중치 순서대로 순서대로)

[('겨울', 0.05626504), ('바다', 0.045322333), ('음악', 0.0376074), ('홀로', 0.031109324), ('파도', 0.03045646), ('여름', 0.029260967), ('멀리', 0.029225405), ('봄날', 0.027252663), ('보이', 0.02493753), ('발자국', 0.022731166)]


In [49]:
# ★ 전체 토픽별 주요 단어 출력 
# print_topic_words(lda_model122)  
# 위에서 이미 살펴본 결과

In [44]:
# ★ 전체 문서의 토픽별 가중치 출력
# 시간 관계상 2번 문서까지만 출력, "0번문서, 1번문서, 2번문서"에 대해서만 해당문서의 토픽가중치를 표시
print_doc_topics(lda_model122, corpus)       

Doc num: 0
	3	0.11654195934534073
	5	0.7920902967453003
	7	0.03748706728219986
	8	0.027153989300131798


Doc num: 1
	1	0.011517386883497238
	3	0.8447270393371582
	5	0.016013139858841896
	7	0.055718839168548584
	8	0.016053669154644012
	11	0.014664120972156525


Doc num: 2
	3	0.4500356614589691
	5	0.36871564388275146
	7	0.12028491497039795
	8	0.011708990670740604
	11	0.010710997506976128


하나의 document에서 너무 많은 토픽으로 단어가 분류되지 않는 모습입니다.

## 결론

지금까지 시대별 멜론 차트 Top 30 데이터를 가지고 LSA, LDA 분석을 진행해 보았습니다. 우선 perplexity와 coherence를 기준으로 몇 개의 토픽을 가지고 나눌 때 가장 효과적일지에 대해 알아보았고, 그 결과 높은 coherence를 기준으로 12개의 토픽으로 데이터를 나눌 때가 가장 잘 나뉘어 진다고 판단하고 진행하였습니다. 실제로 토픽을 나눈 결과를 보았을 때 적절한 비중으로 단어들이 나뉘어져 있었고 뚜렷하지는 않더라도 어느정도 비슷한 단어들이 잘 묶여 있던 것을 확인할 수 있었습니다. 데이터 자체가 시대별로 인기 있던 데이터이기 때문에 주로 이별이나 슬픔, 사랑에 대한 단어들이 많이 포진하고 있었음을 감안할 때 12개의 토픽으로 잘 분류하였다고 생각됩니다. 또한 이를 시각화 해보니 토픽이 29개일 때보다 12개일 때가 더 토픽들이 뭉쳐있지 않고, 전체 문서의 토픽별 가중치를 출력해봐도 하나의 Document에서 딱 몇가지의 토픽으로만 단어들을 보낸 것을 보니 모델이 잘 만들어 졌다고 판단할 수 있습니다. 

반대로 LSA 분석에서는 전체 토픽별 주요 어휘를 추출해본 결과 전체적으로 데이터에서 많이 등장했던 단어들이 모든 토픽에 들어가있어 특정 토픽을 찾기 어려웠습니다. 이를 전체 문서의 토픽별 가중치 분석을 통해 살펴보니 실제로 하나의 Document에서 모든 토픽으로 단어들을 보낸 것을 확인할 수 있었습니다. 따라서 LSA모델을 통한 분석은 해당 멜론 차트 데이터에는 잘 맞지 않다고 판단하였고 위의 결과가 잘 분류된 LDA 모델을 사용하는 것이 낫다고 생각하였습니다.

분류한 토픽과 그에 대한 분석은 윗 부분의 lda_model12 모델을 생성하는 부분에서 확인하실 수 있습니다. 감사합니다.